In [1]:
# ==============================================================================
#                 CELL 1: COMPLETE PROJECT SETUP (FINAL)
# ------------------------------------------------------------------------------
# Purpose: This single cell handles all setup: mounting, paths, libraries,
#          and imports to ensure the environment is correctly configured.
# ==============================================================================
import os
from google.colab import drive

# --- 1. Mounting and Paths ---
print("📁 Mounting Google Drive...")
drive.mount('/content/drive')

# IMPORTANT: Verify this is the correct folder name!
BASE_PROJECT_PATH = '/content/drive/MyDrive/brain-tumor/'
# If your folder is named 'brain-tumor (1)', change the line above to:
# BASE_PROJECT_PATH = '/content/drive/MyDrive/brain-tumor (1)/'

assert os.path.exists(BASE_PROJECT_PATH), f"Error: The path '{BASE_PROJECT_PATH}' does not exist. Please check the folder name."
print(f"✅ Project path verified: {BASE_PROJECT_PATH}")

# --- 2. Install and Import All Libraries ---
print("\n📦 Installing all necessary libraries...")
# FIX: Added streamlit and pyngrok to the installation
!pip install -q ultralytics streamlit pyngrok

import pickle
import numpy as np
import pandas as pd
import torch
import cv2
from ultralytics import YOLO
from collections import Counter
from google.colab import files
import glob

print("✅ All libraries installed and imported successfully.")

📁 Mounting Google Drive...
Mounted at /content/drive
✅ Project path verified: /content/drive/MyDrive/brain-tumor/

📦 Installing all necessary libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ All libraries installed and imported successfully.


In [2]:
# ==============================================================================
#      CELL 2: LOAD TRAINED MODEL AND CALIBRATED PREDICTIONS (CORRECTED)
# ------------------------------------------------------------------------------
# Purpose: Load assets from Phase 2 and define all necessary helper functions.
# ==============================================================================

# --- Load the saved test set predictions ---
try:
    TEST_DATA_PATH = os.path.join(BASE_PROJECT_PATH, 'Project_Clinically_Aware_AI/all_detections_data_TEST.pkl')
    with open(TEST_DATA_PATH, 'rb') as f:
        all_detections_data_TEST = pickle.load(f)
    print(f"✅ Successfully loaded {len(all_detections_data_TEST)} pre-computed detections from the test set.")
except FileNotFoundError:
    print(f"❌ ERROR: Could not find the test data file at '{TEST_DATA_PATH}'. Please ensure the path is correct.")


# --- Helper functions to get the context key ---
CLASS_NAMES = ['glioma', 'meningioma', 'no-tumor', 'pituitary']
TUMOR_SIZE_THRESHOLD = 0.0925

# --- NEW: ALL HELPER FUNCTIONS ARE NOW DEFINED HERE ---

def get_image_sharpness(image_array_grayscale):
    """Calculates image sharpness using the variance of the Laplacian."""
    if image_array_grayscale is None:
        return 0
    return cv2.Laplacian(image_array_grayscale, cv2.CV_64F).var()

def get_box_area(box):
    area = (box[2] - box[0]) * (box[3] - box[1])
    return area / (640 * 640)

def get_box_location(box):
    center_x = (box[0] + box[2]) / 2
    return 'Center' if (640 / 3 < center_x < 2 * 640 / 3) else 'Side'

def get_clinical_context_key(p):
    quality = "HighQuality" if p['sharpness'] > 33.02 else "LowQuality"
    class_name = CLASS_NAMES[p['class_id']].replace('-', '')
    size = "Large" if get_box_area(p['box']) > TUMOR_SIZE_THRESHOLD else "Small"
    location = get_box_location(p['box'])
    return f"{quality}_{class_name}_{size}_{location}"

# --- Re-create the calibrated predictions from your 32-group strategy ---
super_hybrid_temps = {
    'HighQuality_glioma_Small_Center': 0.4279, 'HighQuality_glioma_Small_Side': 1.6735,
    'HighQuality_meningioma_Large_Center': 0.1527, 'HighQuality_meningioma_Large_Side': 0.1985,
    'HighQuality_meningioma_Small_Center': 0.6029, 'HighQuality_meningioma_Small_Side': 0.0026,
    'HighQuality_notumor_Large_Center': 0.4716, 'HighQuality_pituitary_Small_Center': 0.0136,
    'LowQuality_glioma_Small_Center': 0.5259, 'LowQuality_glioma_Small_Side': 0.3803,
    'LowQuality_meningioma_Small_Center': 0.3721, 'LowQuality_meningioma_Small_Side': 0.3564,
    'LowQuality_notumor_Large_Center': 0.8852, 'LowQuality_pituitary_Small_Center': 0.4192
}

calibrated_predictions = []
for p in all_detections_data_TEST:
    context_key = get_clinical_context_key(p)
    temperature = super_hybrid_temps.get(context_key, 1.0)
    calibrated_conf = 1 / (1 + np.exp(-p['logit'] / temperature))
    final_prediction = p.copy()
    final_prediction['calibrated_confidence'] = calibrated_conf
    final_prediction['class_name'] = CLASS_NAMES[p['class_id']]
    calibrated_predictions.append(final_prediction)

print(f"✅ Generated {len(calibrated_predictions)} final calibrated predictions using your 32-group strategy.")

✅ Successfully loaded 262 pre-computed detections from the test set.
✅ Generated 262 final calibrated predictions using your 32-group strategy.


In [3]:
# ==============================================================================
#                 CELL 3: DEFINE THE TRIAGE LOGIC
# ------------------------------------------------------------------------------
# Purpose: To define the rules-based engine that sorts cases into
#          clinical priority queues.
# ==============================================================================

def get_triage_category(prediction):
    """
    Sorts a single prediction into a clinical triage queue.

    Args:
        prediction (dict): A dictionary containing the calibrated prediction info,
                           including 'calibrated_confidence' and 'class_name'.

    Returns:
        tuple: A tuple containing the queue name (str) and a color code (str).
    """
    confidence = prediction['calibrated_confidence']
    class_name = prediction['class_name']

    # Rule 1: High-confidence "No-Tumor" cases are safe to fast-track
    if class_name == 'no-tumor' and confidence > 0.90:
        return "Green Queue (Routine Review)", "Green"

    # Rule 2: High-confidence tumor detections are urgent
    if class_name != 'no-tumor' and confidence > 0.90:
        return "Red Queue (Urgent Review)", "Red"

    # Rule 3: All other (lower confidence) cases require standard review
    return "Yellow Queue (Standard Review)", "Yellow"

print("✅ Triage logic function is defined and ready to use.")

# --- Test the function with a sample prediction ---
sample_prediction = calibrated_predictions[0] # Take the first prediction from our list
triage_queue, color = get_triage_category(sample_prediction)

print("\n--- Testing with a sample case ---")
print(f"Class: {sample_prediction['class_name']}, Calibrated Confidence: {sample_prediction['calibrated_confidence']:.3f}")
print(f"Assigned Triage Status: {triage_queue}")

✅ Triage logic function is defined and ready to use.

--- Testing with a sample case ---
Class: pituitary, Calibrated Confidence: 1.000
Assigned Triage Status: Red Queue (Urgent Review)


In [4]:
# ==============================================================================
#                 CELL 4: TRIAGE SIMULATION ON THE FULL TEST SET
# ------------------------------------------------------------------------------
# Purpose: To apply our triage logic to the entire test set and quantify
#          its potential impact on clinical workflow.
# ==============================================================================
from collections import Counter
import pandas as pd

print("🚀 Running full triage simulation on the test set...")

# A counter to store the number of cases in each queue
triage_counts = Counter()

# Lists to hold a few examples from each queue for review
green_examples = []
yellow_examples = []
red_examples = []

# Loop through every calibrated prediction from our test set
for prediction in calibrated_predictions:
    triage_queue, color = get_triage_category(prediction)

    # Increment the count for the assigned queue
    triage_counts[triage_queue] += 1

    # Save a few examples for inspection later
    if color == "Green" and len(green_examples) < 3:
        green_examples.append(prediction)
    elif color == "Yellow" and len(yellow_examples) < 3:
        yellow_examples.append(prediction)
    elif color == "Red" and len(red_examples) < 3:
        red_examples.append(prediction)

# --- Quantitative Analysis ---
total_predictions = len(calibrated_predictions)
results = []
for queue, count in triage_counts.items():
    percentage = (count / total_predictions) * 100
    results.append({
        "Triage Queue": queue,
        "Number of Cases": count,
        "Percentage of Workload": f"{percentage:.2f}%"
    })

results_df = pd.DataFrame(results)

print("\n" + "="*60)
print("--- 📊 TRIAGE SIMULATION RESULTS ---")
print("="*60)
print(results_df.to_string(index=False))

# --- Qualitative Analysis: Showcasing Examples ---
print("\n" + "="*60)
print("--- 📝 EXAMPLE CASE ASSIGNMENTS ---")
print("="*60)

print("\n🔴 Example of a 'Red Queue (Urgent Review)' case:")
if red_examples:
    p = red_examples[0]
    print(f"   - Predicted Class: {p['class_name']}")
    print(f"   - Calibrated Confidence: {p['calibrated_confidence']:.3f}")
else:
    print("   - No cases were assigned to the Red Queue in this run.")

print("\n🟡 Example of a 'Yellow Queue (Standard Review)' case:")
if yellow_examples:
    p = yellow_examples[0]
    print(f"   - Predicted Class: {p['class_name']}")
    print(f"   - Calibrated Confidence: {p['calibrated_confidence']:.3f}")
else:
    print("   - No cases were assigned to the Yellow Queue in this run.")

print("\n🟢 Example of a 'Green Queue (Routine Review)' case:")
if green_examples:
    p = green_examples[0]
    print(f"   - Predicted Class: {p['class_name']}")
    print(f"   - Calibrated Confidence: {p['calibrated_confidence']:.3f}")
else:
    print("   - No cases were assigned to the Green Queue in this run.")

🚀 Running full triage simulation on the test set...

--- 📊 TRIAGE SIMULATION RESULTS ---
                  Triage Queue  Number of Cases Percentage of Workload
     Red Queue (Urgent Review)              134                 51.15%
Yellow Queue (Standard Review)               95                 36.26%
  Green Queue (Routine Review)               33                 12.60%

--- 📝 EXAMPLE CASE ASSIGNMENTS ---

🔴 Example of a 'Red Queue (Urgent Review)' case:
   - Predicted Class: pituitary
   - Calibrated Confidence: 1.000

🟡 Example of a 'Yellow Queue (Standard Review)' case:
   - Predicted Class: glioma
   - Calibrated Confidence: 0.881

🟢 Example of a 'Green Queue (Routine Review)' case:
   - Predicted Class: no-tumor
   - Calibrated Confidence: 0.975


In [5]:
# ==============================================================================
#                      NEW CELL 5: DEBUG LAUNCHER
# ------------------------------------------------------------------------------
# Purpose: To run the Streamlit app in the foreground to expose any
#          hidden startup errors and their tracebacks.
# ==============================================================================

!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.205.128:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1363, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1830, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1226, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [6]:
# ==============================================================================
#                 CELL 6: DEPLOY APP WITH NGROK (ROBUST METHOD)
# ------------------------------------------------------------------------------
# Purpose: To reliably launch the Streamlit app using ngrok, which is
#          more stable than Colab's default external URL.
# ==============================================================================

# --- 1. Install pyngrok ---
!pip install -q pyngrok

# --- 2. Paste your ngrok authtoken here ---
# Replace the placeholder text with the token you copied from your ngrok dashboard.
NGROK_AUTH_TOKEN = "32odaMtz4LNdW98afCIIR99PXY1_6Fiy6CUW2TeU5RqQbcafX"

# --- 3. The Code to Launch the App ---
import os
import threading
import time
from pyngrok import ngrok

# Authenticate ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Kill any existing streamlit or ngrok processes
!killall streamlit
!killall ngrok
time.sleep(2)

# Function to run streamlit in the background
def run_streamlit():
    os.system("streamlit run app.py --server.headless true --server.enableCORS false")

# Start streamlit in a new thread
thread = threading.Thread(target=run_streamlit)
thread.start()
time.sleep(5) # Give streamlit a moment to start

# Open a tunnel to the streamlit port (8501)
public_url = ngrok.connect(8501)
print("="*50)
print("✅ Your app is now live and running!")
print(f"🚀 Click this Public URL to open your app: {public_url}")
print("="*50)

streamlit: no process found
ngrok: no process found
✅ Your app is now live and running!
🚀 Click this Public URL to open your app: NgrokTunnel: "https://a1c2acbc5c2a.ngrok-free.app" -> "http://localhost:8501"


In [7]:
# ==============================================================================
#                 CELL 7: FIND AND DOWNLOAD TEST IMAGES
# ------------------------------------------------------------------------------
# Purpose: To automatically find ideal images for each triage test case
#          and download them from Google Drive to your local computer.
# ==============================================================================
from google.colab import files
import glob

print("🔎 Searching for ideal test case images in your validation set...")

# --- 1. Define search parameters and the image directory ---
VALIDATION_IMAGE_DIR = os.path.join(BASE_PROJECT_PATH, 'Project_Clinically_Aware_AI/dataset_setup/validation_images/')

# We will store the paths to the best images we find
image_paths_to_download = {}

# --- 2. Iterate through files to find our test cases ---
# We'll grab all .jpg files from the directory
all_images = glob.glob(os.path.join(VALIDATION_IMAGE_DIR, "*.jpg"))

for image_path in all_images:
    filename = os.path.basename(image_path)

    # We need to calculate sharpness to find high/low quality images
    # The get_image_sharpness function is already in memory from our setup
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sharpness = get_image_sharpness(img_gray)

    # Test Case 1: Find a high-quality "no-tumor" scan for the Green Queue
    if "green_queue_test" not in image_paths_to_download and filename.startswith("Tr-no_") and sharpness > 100:
        image_paths_to_download["green_queue_test.jpg"] = image_path
        print(f"  - Found Green Queue example: {filename} (Sharpness: {sharpness:.2f})")

    # Test Case 2: Find a clear, high-quality "meningioma" for the Red Queue
    if "red_queue_test" not in image_paths_to_download and filename.startswith("Tr-me_") and sharpness > 100:
        image_paths_to_download["red_queue_test.jpg"] = image_path
        print(f"  - Found Red Queue example: {filename} (Sharpness: {sharpness:.2f})")

    # Test Case 3: Find a low-quality "glioma" for the Yellow Queue
    if "yellow_queue_test" not in image_paths_to_download and filename.startswith("Tr-gl_") and sharpness < 40:
        image_paths_to_download["yellow_queue_test.jpg"] = image_path
        print(f"  - Found Yellow Queue example: {filename} (Sharpness: {sharpness:.2f})")

    # Stop searching once we have found one of each
    if len(image_paths_to_download) == 3:
        break

# --- 3. Download the selected files to your local machine ---
print("\n⬇️ Preparing to download the 3 test images to your computer...")

if not image_paths_to_download:
    print("⚠️ Could not find suitable images for all test cases. Please check the directory path.")
else:
    for new_name, original_path in image_paths_to_download.items():
        # Copy the file to the local Colab session with a simpler name first
        !cp "{original_path}" ./{new_name}
        # Trigger the browser download
        files.download(new_name)
    print("\n✅ Download complete! Check your browser's download bar.")

🔎 Searching for ideal test case images in your validation set...
  - Found Yellow Queue example: Tr-gl_0011_jpg.rf.c6586a6320264a828625b45ba88fabac.jpg (Sharpness: 34.90)
  - Found Yellow Queue example: Tr-gl_0302_jpg.rf.0d359f595d6641267c9dfcec1c97c4ae.jpg (Sharpness: 30.19)
  - Found Yellow Queue example: Tr-gl_0310_jpg.rf.55b2520296336ec2735cb44f5650d53d.jpg (Sharpness: 13.93)
  - Found Yellow Queue example: Tr-gl_0221_jpg.rf.2d2ec67a535c9b74acc29713603d76c0.jpg (Sharpness: 12.85)
  - Found Yellow Queue example: Tr-gl_0057_jpg.rf.00db06a404574ba14346d9a5416f878f.jpg (Sharpness: 19.51)
  - Found Yellow Queue example: Tr-gl_0175_jpg.rf.c61d1aed455e8185a1e8925785b4480a.jpg (Sharpness: 12.62)
  - Found Yellow Queue example: Tr-gl_0015_jpg.rf.f19712315a693c8362217ca980843f70.jpg (Sharpness: 30.90)
  - Found Yellow Queue example: Tr-gl_0014_jpg.rf.b717ae199e12230bf44b56eba9d741b1.jpg (Sharpness: 38.26)
  - Found Yellow Queue example: Tr-gl_0018_jpg.rf.32814a3e780f52883d520898ecedfb4c.jpg 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download complete! Check your browser's download bar.


In [8]:
# ==============================================================================
#                 CELL 8: CREATE COMPREHENSIVE TEST SET
# ------------------------------------------------------------------------------
# Purpose: To find and download 12 diverse images, including out-of-distribution
#          metastasis cases, for a final, rigorous system test.
# ==============================================================================
from google.colab import files
import glob
import random

print("🔎 Assembling a comprehensive test set of 12 images...")

# --- 1. Define paths to both datasets ---
VALIDATION_IMAGE_DIR = os.path.join(BASE_PROJECT_PATH, 'Project_Clinically_Aware_AI/dataset_setup/validation_images/')
# NOTE: This path assumes the challenge/metastasis data was processed into this folder structure.
# If you have it elsewhere, you may need to adjust this path.
METASTASIS_IMAGE_DIR = os.path.join(BASE_PROJECT_PATH, 'Challenge_processed_2D/images/')

# --- 2. Define the list of images we want to find ---
image_requests = [
    {'name': 'High-Quality_No-Tumor', 'folder': VALIDATION_IMAGE_DIR, 'prefix': 'Tr-no_', 'quality': 'high', 'count': 2},
    {'name': 'Low-Quality_No-Tumor',  'folder': VALIDATION_IMAGE_DIR, 'prefix': 'Tr-no_', 'quality': 'low',  'count': 1},
    {'name': 'High-Quality_Meningioma','folder': VALIDATION_IMAGE_DIR, 'prefix': 'Tr-me_', 'quality': 'high', 'count': 2},
    {'name': 'Low-Quality_Glioma',    'folder': VALIDATION_IMAGE_DIR, 'prefix': 'Tr-gl_', 'quality': 'low',  'count': 2},
    {'name': 'High-Quality_Pituitary','folder': VALIDATION_IMAGE_DIR, 'prefix': 'Tr-pi_', 'quality': 'high', 'count': 2},
    {'name': 'Metastasis_Tumor',      'folder': METASTASIS_IMAGE_DIR, 'prefix': '',       'quality': 'any',  'count': 3},
]

# --- 3. The Search Engine ---
found_images = []
for req in image_requests:
    print(f"  - Searching for {req['count']}x {req['name']}...")

    # Get all files from the target folder, shuffling them for variety
    all_files_in_folder = glob.glob(os.path.join(req['folder'], "*.jpg"))
    random.shuffle(all_files_in_folder)

    found_count = 0
    for image_path in all_files_in_folder:
        if found_count >= req['count']:
            break # Stop once we've found enough images for this request

        filename = os.path.basename(image_path)

        if req['prefix'] and not filename.startswith(req['prefix']):
            continue # Skip if the prefix doesn't match

        # Check image quality
        img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        sharpness = get_image_sharpness(img_gray)

        passes_quality_check = False
        if req['quality'] == 'high' and sharpness > 100:
            passes_quality_check = True
        elif req['quality'] == 'low' and sharpness < 40:
            passes_quality_check = True
        elif req['quality'] == 'any':
            passes_quality_check = True

        if passes_quality_check:
            # Create a descriptive new name for the downloaded file
            new_filename = f"test_{len(found_images) + 1}_{req['name']}.jpg"
            found_images.append({'new_name': new_filename, 'path': image_path})
            found_count += 1

# --- 4. Download all the found files ---
print(f"\n⬇️ Found {len(found_images)} total images. Preparing to download...")

if not found_images:
    print("⚠️ Could not find any suitable images. Please check the directory paths.")
else:
    for image_info in found_images:
        !cp "{image_info['path']}" ./"{image_info['new_name']}"
        files.download(image_info['new_name'])
    print(f"\n✅ {len(found_images)} images downloaded! Check your browser's download bar.")

🔎 Assembling a comprehensive test set of 12 images...
  - Searching for 2x High-Quality_No-Tumor...
  - Searching for 1x Low-Quality_No-Tumor...
  - Searching for 2x High-Quality_Meningioma...
  - Searching for 2x Low-Quality_Glioma...
  - Searching for 2x High-Quality_Pituitary...
  - Searching for 3x Metastasis_Tumor...

⬇️ Found 8 total images. Preparing to download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 8 images downloaded! Check your browser's download bar.


In [9]:
# ==============================================================================
#                 CELL 9: IMPROVEMENT 1 - DYNAMIC THRESHOLDS
# ------------------------------------------------------------------------------
# Purpose: To implement and simulate a more nuanced triage system using
#          different confidence thresholds for each prediction class.
# ==============================================================================
from collections import Counter
import pandas as pd

print("🚀 Implementing Improvement 1: Dynamic, Class-Specific Thresholds...")

# --- 1. Define the new, clinically-nuanced thresholds ---
# These are our new rules, reflecting different levels of caution.
triage_thresholds = {
    'no-tumor': 0.95,    # We must be very certain to classify a scan as normal.
    'meningioma': 0.90,  # A common, often less aggressive tumor.
    'pituitary': 0.92,   # Often well-defined, so we can require high confidence.
    'glioma': 0.88       # Can be diffuse and ambiguous, so we flag it even at a lower confidence.
}
print("   - New thresholds defined.")

# --- 2. Create the new, more advanced triage function ---
def get_triage_category_dynamic(prediction, thresholds):
    """Sorts a prediction into a queue using class-specific thresholds."""
    confidence = prediction['calibrated_confidence']
    class_name = prediction['class_name']

    # Get the specific threshold for the predicted class
    threshold = thresholds.get(class_name, 0.90) # Default to 0.90 if class is unknown

    if confidence > threshold:
        if class_name == 'no-tumor':
            return "Green Queue (Routine Review)", "Green"
        else:
            return "Red Queue (Urgent Review)", "Red"
    else:
        return "Yellow Queue (Standard Review)", "Yellow"

print("   - New dynamic triage function defined.")

# --- 3. Run the new simulation ---
print("🚀 Running new simulation with dynamic thresholds...")
dynamic_triage_counts = Counter()

for prediction in calibrated_predictions:
    triage_queue, _ = get_triage_category_dynamic(prediction, triage_thresholds)
    dynamic_triage_counts[triage_queue] += 1

# --- 4. Display the new results ---
total_predictions = len(calibrated_predictions)
results = []
for queue, count in dynamic_triage_counts.items():
    percentage = (count / total_predictions) * 100
    results.append({
        "Triage Queue": queue,
        "Number of Cases": count,
        "Percentage of Workload": f"{percentage:.2f}%"
    })

results_df = pd.DataFrame(results)

print("\n" + "="*60)
print("--- 📊 DYNAMIC THRESHOLD SIMULATION RESULTS ---")
print("="*60)
print(results_df.to_string(index=False))

🚀 Implementing Improvement 1: Dynamic, Class-Specific Thresholds...
   - New thresholds defined.
   - New dynamic triage function defined.
🚀 Running new simulation with dynamic thresholds...

--- 📊 DYNAMIC THRESHOLD SIMULATION RESULTS ---
                  Triage Queue  Number of Cases Percentage of Workload
     Red Queue (Urgent Review)              137                 52.29%
Yellow Queue (Standard Review)              106                 40.46%
  Green Queue (Routine Review)               19                  7.25%


In [10]:
# ==============================================================================
#            CELL 10: IMPROVEMENT 2 - CONTINUOUS URGENCY SCORE
# ------------------------------------------------------------------------------
# Purpose: To implement a sophisticated scoring system that calculates a
#          continuous urgency score based on multiple clinical factors.
# ==============================================================================
import pandas as pd

print("🚀 Implementing Improvement 2: Continuous Clinical Urgency Score...")

# --- 1. Define the weights for our scoring formula ---
# These weights determine the importance of each factor.
urgency_weights = {
    'confidence': 1.0,   # Higher confidence increases urgency (for tumors).
    'sharpness': -0.001, # Poor quality (low sharpness) slightly increases urgency.
    'size': 0.5          # Larger tumor size increases urgency.
}
print("   - Urgency weights defined.")

# --- 2. Create the new scoring function ---
def calculate_urgency_score(prediction, weights):
    """Calculates a raw urgency score based on a weighted formula."""

    # We only calculate urgency for potential tumors. Normal scans get a score of 0.
    if prediction['class_name'] == 'no-tumor':
        return 0.0

    # Extract features
    confidence = prediction['calibrated_confidence']
    sharpness = prediction['sharpness']
    size = get_box_area(prediction['box']) # get_box_area is defined in Cell 2

    # Calculate the raw weighted score
    raw_score = (confidence * weights['confidence'] +
                 sharpness * weights['sharpness'] +
                 size * weights['size'])

    return raw_score

print("   - Urgency score function defined.")

# --- 3. Run the new simulation ---
print("🚀 Calculating urgency scores for all test set cases...")
all_scores = []
for p in calibrated_predictions:
    raw_score = calculate_urgency_score(p, urgency_weights)
    all_scores.append({
        'class_name': p['class_name'],
        'raw_score': raw_score,
        'calibrated_confidence': p['calibrated_confidence'],
        'sharpness': p['sharpness'],
        'size': get_box_area(p['box'])
    })

# --- 4. Analyze the results ---
scores_df = pd.DataFrame(all_scores)

# To scale the scores from 1-100, we first filter out the "no-tumor" cases
tumor_scores = scores_df[scores_df['class_name'] != 'no-tumor']['raw_score']
min_score = tumor_scores.min()
max_score = tumor_scores.max()

# Scale the raw scores to a 1-100 range for easier interpretation
scores_df['Urgency Score (1-100)'] = scores_df['raw_score'].apply(
    lambda x: 1 + 99 * (x - min_score) / (max_score - min_score) if x > 0 else 0
)

print("\n" + "="*70)
print("--- 📊 URGENCY SCORE SIMULATION RESULTS ---")
print("="*70)
print("Summary statistics for the calculated Urgency Score (for tumor cases only):")
# Show summary stats for the final scaled scores of actual tumors
print(scores_df[scores_df['class_name'] != 'no-tumor']['Urgency Score (1-100)'].describe().round(2))

print("\n--- 📝 EXAMPLE CASE SCORES ---")
# Display the cases with the HIGHEST and LOWEST urgency scores
highest_urgency_cases = scores_df.nlargest(3, 'Urgency Score (1-100)')
lowest_urgency_cases = scores_df[scores_df['class_name'] != 'no-tumor'].nsmallest(3, 'Urgency Score (1-100)')

print("\n⬆️ Top 3 Highest Urgency Cases:")
print(highest_urgency_cases[['class_name', 'Urgency Score (1-100)']].round(2).to_string(index=False))

print("\n⬇️ Top 3 Lowest Urgency (but still tumor) Cases:")
print(lowest_urgency_cases[['class_name', 'Urgency Score (1-100)']].round(2).to_string(index=False))

🚀 Implementing Improvement 2: Continuous Clinical Urgency Score...
   - Urgency weights defined.
   - Urgency score function defined.
🚀 Calculating urgency scores for all test set cases...

--- 📊 URGENCY SCORE SIMULATION RESULTS ---
Summary statistics for the calculated Urgency Score (for tumor cases only):
count    209.00
mean      83.81
std       23.43
min        0.00
25%       82.39
50%       94.89
75%       97.17
max      100.00
Name: Urgency Score (1-100), dtype: float64

--- 📝 EXAMPLE CASE SCORES ---

⬆️ Top 3 Highest Urgency Cases:
class_name  Urgency Score (1-100)
meningioma                 100.00
meningioma                  99.79
meningioma                  99.75

⬇️ Top 3 Lowest Urgency (but still tumor) Cases:
class_name  Urgency Score (1-100)
 pituitary                    0.0
 pituitary                    0.0
meningioma                    0.0
